# 04 NetCDF Data - How to plot it

Now we have all the tools to get to the most motivating part of this whole book: learn, how to plot the data and get the output!

Xarray plotting functionality is a thin wrapper around the popular **matplotlib library**. If you are not familiar with any kind of plotting tool, you can have a look at the [matplotlib tutorial](https://matplotlib.org/tutorials/introductory/pyplot.html#sphx-glr-tutorials-introductory-pyplot-py)! It very well explains the basics of plotting. 

Matplotlib syntax and function names were copied as much as possible for xarray plotting, which makes for an easy transition between the two. Therefore you should be able, to plot xarray data, once you learned how to plot with matplotlib. Anyway, we will quickly go through the basics here as well! Matplotlib must be installed and imported, before xarray can plot.

In [ ]:
# Display the plots in the notebook:
%matplotlib inline

In [ ]:
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary, needed to display world maps
import cartopy.crs as ccrs  # Projections list
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen
# The commands below are to ignore certain warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ds = xr.open_dataset('./data_final/ERA5-LowRes-MonthlyAvg-t2m_tp.nc') #open the t2m + precipitation dataset
ds

We imported matplotlib as `plt`. From now on whenever we need matplotlib, we will call `plt.`!


## 04_01 Introduction to Plotting

First, we will need to understand how a plot of the matplotlib library is structured. 

From [Real Python:](https://realpython.com/python-matplotlib-guide/#the-matplotlib-object-hierarchy) One important  matplotlib concept is its object hierarchy.  A “hierarchy” here means that there is a tree-like structure of matplotlib objects underlying each plot. 

A **Figure object** is the outermost container for a matplotlib graphic, which can contain multiple **Axes objects**. One source of confusion is the name: an Axes actually translates into what we think of as an individual plot or graph (rather than the plural of “axis,” as we might expect).

You can think of the Figure object as a box-like container holding one or more Axes (actual plots). Below the Axes in the hierarchy are smaller objects such as tick marks, individual lines, legends, and text boxes. Almost every “element” of a chart is its own manipulable Python object, all the way down to the ticks and labels:

![](hierarchy1.jpg "")

Each of those matplotlib objects has an identifier, called handle. You create the handle when you create the actual object. With this handle, you can access the figure/axes you want to and change its properties!

You can now create a figure of a specified size:

In [ ]:
h = plt.figure('figsize', [10,10]) # [width, height] in inches

`h` is the handle for the figure with the size of 10x10 inches. 

Or directly create an axes (a plot!), which creates a figure as well. 

In [ ]:
ax = plt.axes(); # creates an empty plot! 

You could now specify properties for this plot. Those properties will be stored in the handle of the axes: `ax`. You can either set new properties by tiping `ax.set_` + `tab`(all the properties you can set will show up) or have a look at the properties your axes already owns by `ax.get_` + `tab`. 

In [ ]:
ax.set_title('My first axes!') #set the title of our axes
ax.get_title() #see, what the title of 'ax' is??

You can also create several axes, which means you create several subplots! This is done by `plt.subplot(nrows, ncolumns, index)`. 

In [ ]:
ax1 = plt.subplot(2,1,1) # 2 rows, 1 column, this is the first axes
ax2 = plt.subplot(2,1,2) # 2 rows, 1 column, this is the second axes

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1) # this gives the same result!

## 04_02 One Dimension

The simplest way to make a plot is to call the `xarray.DataArray.plot()` method.  Let's do this for the zonal mean of the 2m temperature! Python creates an axes for this plot internally, even though we did not create one on our own!

In [ ]:
ds.t2m.mean(dim = ('time','longitude')).plot();

As we see, the labeling nature of an xarray comes in handy for plotting! We did not give any specifications for the axes' labels and still we get a satisfying labeling. Xarray takes the metadata stored in `attributes`, to label the axes: Latitude was the "long_name" attribute, [degrees_north] the "units" attribute of `ds.lat`, `t2m` the variable name! 

You can also plot **multiple lines** into one figure:

In [ ]:
grouped = ds.t2m.groupby('time.month').mean().mean(dim = 'longitude'); #group the dataset by months and average over lon
grouped.sel(month = 1).plot(); # select month 1, plot it
grouped.sel(month = 7).plot();

Although matplotlib already does quite a good job in adjusting the plot, there are still a couple of problems we have to deal with:

What do those lines stand for?! Why is the title of the plot "month = 7"? If you didn't plot this on your own and knew what you saw, the graphic would be quite useless...

Let's improve the above plot a bit! By doing this, we will learn some basic features of a plot:

First, we want to add a **title**. Notice: we imported matplotlib.pyplot as plt! So we can now work with our plotting library as `plt`. The library comes with all kinds off necessary stuff and therefore it also has a method `plt.title('your_title_here')`!

More editing:

Define **x/y - Limits**: `plt.xlim(startvalue, stopvalue)`

Change the **x/y - Label**: `plt.ylabel('your_label_name_here')`

Change the **x/y - Ticks**: `plt.xticks(ticks, [labels])`. Ticks: A list of positions at which ticks should be placed. You can pass an empty list to disable xticks. [labels]: A list of explicit labels to place at the given locs. Those are optional, so you do not need to specify them. 

Create a **legend** (if there are multiple lines in one plot): `plt.legend()`. The plot will automatically generate a legend for you. If you wish to give other names to your lines, you will need to specify them by giving the `label = 'your_labelname_here'` argument to the plotting method itself. You can change the location of the legend in your plot by giving the optional argument `loc = 'your_location_here'` to the method. Possible locations are e.g.: 'best' (the library will automatically search for a spot in your plot, where there is room for the legend), 'upper right', 'lower left', 'right', 'center', 'lower center', 'center left' and so on. Have a look at the [legend documentation](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.legend.html#matplotlib.pyplot.legend) if you want to get more control on your legends.

Change the **style of your line**: By giving the following optional arguments to your `.plot()` method.
* **LineStyle**: How your line should look like. Possible values: `-` Solid line, `--` Dashed line, `:` Dotted line, `-.` Dash-dot line. Default = `'-'`.
* **LineWidth**: Default = 0.5 points, with point = 1/72 inch = 0.351 mm).
* **Color**: Color of the line (given as RGB vector oder as String). Possible string values: `k` black, `r` red, `g` green, `b` blue, `c` cyan, `m` magenta, `y` yellow, `w` white
* **Marker**: Markertype. Possible Markertypes e.g.: `+`, `o`, `*`, `.`, `x`, `s` (square), `d` (diamond), `p` (five pointed star). Default = `none`.
* **MarkerSize**: Size of marker in points. Default = 6 points.
* **MarkerEdgeColor**: Color of marker border.
* **MarkerFaceColor**: Color of marker body.

You can also insert a **grid** into your plot: `plt.grid()`. Change the style of your grid lines by specifying the same style keywords as explained for the lineplot above!

Add a **reference line** for better orientation to the plot: `plt.axhline(y)` adds a horizontal line across the axis. y is the position in data coordinates of the horizontal line, by default this is 0. You can add further, optional arguments such as `color`, `linestyle` or `linewidth`, etc. They work in the same manner as explained above.

We could have also set each of those properties via the `ax.set` syntax that we learned earlier. Anyway, this is an easier way to get the same result: By directly plotting some data, python automatically created an axes for the plot. Since we define our properties directly underneath the plotting command, python knows that we would like to set the properties of the axes we just created, altough we do not concretely formulate that! 

Later, when we have more complicated looking figures that include a couple of subplots as well, it might be easier to use the `ax.set` syntax in oder to avoid a mess.

Let's try the new features on the plot from above!

In [ ]:
grouped = grouped - 273.15
grouped.sel(month = 1).plot(label = 'January', color = 'c', linestyle = '--') # plots the Jan line with all settings
grouped.sel(month = 7).plot(label = 'July', color = 'r', linestyle = '--')
grouped.mean(dim = 'month').plot(label = 'Annual Avg', linewidth=3, color = 'g') #plot annual avg with a thicker line
plt.xlim(-90, 90) # defines the limits on the x-axis
plt.title('2m Temperature - zonal average') # plots your title
plt.legend(loc='best') # plots your legend defined by the "label"-specification of each plot-command
                       # at the location that fits best for this plot
plt.ylabel('° Celcius'); # changes the y-label from the default label to the one you want
plt.xlabel('Latitude [° North]');
plt.xticks((-90, -60, -30, 0, 30, 60, 90));
plt.axhline(0, color = 'k');
plt.grid(linestyle = '-.')

That looks way better! Now we know, what the lines stand for (legend), what is actually plotted (title), and what our axes are. Plus we made the plot nice looking and added an annual average for orientation!

## 04_03 Two Dimensions - Maps

If we want to plot in two dimensions, the most obvious plot would be our data on a map! E.g. the average temperature:


In [ ]:
t2tavg = ds.t2m.mean(dim = 'time')

In [ ]:
# Define the map projection
ax = plt.axes(projection=ccrs.Robinson())
# ax is an empty plot. We now plot the variable sw_avg onto ax
t2tavg.plot(ax = ax, transform=ccrs.PlateCarree()) 
# the keyword "transform" tells the function in which projection the data is stored 
ax.coastlines(); ax.gridlines(); # Add gridlines and coastlines to the plot

What did we just do? Let's go through each step seperately. 

First, we created an axes object called `ax` just as before. The only new thing: we defined the projection of this axes! Therefore we used the cartopy library. From the [cartopy webpage](https://scitools.org.uk/cartopy/docs/latest/): Cartopy is a Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses. Key functions of cartopy are its object oriented projection definitions, and its ability to transform points, lines, vectors, polygons and images between those projections. 
[Here](https://scitools.org.uk/cartopy/docs/latest/crs/projections.html), you can find a list of all the possible projections! 

Some information about the Robinson Projection, that we used above: 
The robinson projection is pseudocylindrical, and a compromise that is neither equal-area nor conformal. Conformal means, that every angle between two curves that cross each other on Earth is preserved in the image of the projection (e.g. if two streets cross each other at an angle 90°, the angle on the map will also be 90°!).

The robinson projection is commonly used for “visually-appealing” world maps. We will most of the time use this type of projection. 

Anyway, we can try some others too!



### 04_03_01 Map Projections

E.g. `Mollweide`: This projection is pseudocylindrical, and equal area. Parallels are unequally-spaced straight lines, while meridians are elliptical arcs up to semicircles on the edges. Poles are points.
It is commonly used for world maps, or interrupted with several central meridians.

In [ ]:
# Define the map projection
ax = plt.axes(projection=ccrs.Mollweide())
# ax is an empty plot. We now plot the variable sw_avg onto ax
t2tavg.plot(ax=ax, transform=ccrs.PlateCarree()) 
# the keyword "transform" tells the function in which projection the data is stored 

ax.coastlines(); ax.gridlines(); # Add gridlines and coastlines to the plot

The projection `PlateCarree` returns a map, that maps meridians to vertical straight lines of constant spacing (for meridional intervals of constant spacing), and circles of latitude to horizontal straight lines of constant spacing (for constant intervals of parallels). The projection is neither equal area nor conformal. The distortion gets bigger, the closer we get to the equator.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
t2tavg.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines();

In plots as the two above ones, the south and north pole are not represented realistically. So if we want to have a closer look at regions such as the antarctica, we need to change our projection. `SouthPolarStereo()` or `NorthPolarStereo()` are the projections, that look from top directly on the South/Northpole.

In [ ]:
# Prepare the figure with the wanted size:
fig = plt.figure(figsize=(9, 7))
ax = plt.axes(projection=ccrs.SouthPolarStereo())
ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree()) #explanation underneath!
t2tavg.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines();
ax.gridlines();

In order to make it a nice looking plot, we need to specify the extent of our axes (confine it to the antarctica) and change our figure size. To specify the extent: `ax.set_extent([lon_min, lon_max,lat_min,lat_max],projection_of_the_data)`.

There is also a special projection, to display europe as correctly as possible: `EuroPP`. 

In [ ]:
ax = plt.axes(projection=ccrs.EuroPP())
t2tavg.plot(ax=ax, transform=ccrs.PlateCarree(), vmin = 295, vmax= 305)
plt.title('Europe')
ax.coastlines(); ax.gridlines();

Let's get back to our first map plot: What did we do next?

Second, we plotted our data. We specified two optional (**keyword**) arguments of the plot method: We defined our axes as the one we had just created above. Again here the result would have been the same without specifying the axes, since python would have known that we want to use the above created axes. Anyway, with more axes, things will get messy and so we made sure that python knows on which axes we want to plot by giving the keyword argument `ax=ax`. The second keyword argument we specified is the `transform` keyword. This tells the function in which projection the data is stored, in the case [ERA5 data](https://confluence.ecmwf.int/display/CKB/ERA5%3A+What+is+the+spatial+reference): PlateCarree. Now the function knows, that it will have to transform our data from the PlateCarree Projection into the Robinson Projection in order to make everything work. 

Third, we added coastlines (to be able to distinguish between land and ocean) and gridlines (latitude and longitude lines for better orientation). 

### 04_03_02 Map Options

Let's take our Europe Projection once more and try some map options on it!

In [ ]:
ax = plt.axes(projection=ccrs.EuroPP())
t2tavg.plot(ax=ax, transform=ccrs.PlateCarree(), vmin = 295, vmax= 305)
plt.title('Europe')
ax.coastlines();

We already applied a first map option here: we added **coastlines** to be able to distinguish between land and ocean. We can change the color of those coastlines and furthermore, we can also add **country borders**:

In [ ]:
ax = plt.axes(projection=ccrs.EuroPP())
t2tavg.plot(ax=ax, transform=ccrs.PlateCarree(), vmin = 295, vmax= 305)
plt.title('Europe')
ax.coastlines(color = 'pink'); 
ax.add_feature(cartopy.feature.BORDERS);

The borders are a pre-defined feature of the cartopy library! Therefore it is stored in `cartopy.feature.BORDERS` By calling the `ax.add_feature()` method, you can add any kind of feature to your plot (a feature is a collection of points, lines and polygons). 

Next, we can add **gridlines** to the plot. 

In [ ]:
ax = plt.axes(projection=ccrs.EuroPP())
t2tavg.plot(ax=ax, transform=ccrs.PlateCarree(), vmin = 295, vmax= 305)
plt.title('Europe')
ax.coastlines(); 
ax.gridlines();

If we use the PlateCarree or the Mercator projection, we can label those gridlines: 

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
t2tavg.plot(ax=ax, transform=ccrs.PlateCarree(), vmin = 250, vmax= 305)
plt.title('Europe')
ax.coastlines(); 
ax.gridlines(draw_labels = True); #labels the gridlines!

We do not want the labeling on all four sides of the plot? We can turn them off each seperately!


In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
t2tavg.plot(ax=ax, transform=ccrs.PlateCarree(), vmin = 250, vmax= 305)
plt.title('Europe')
ax.coastlines(); 
grid = ax.gridlines(draw_labels = True); 
grid.ylabels_left = False;
grid.xlabels_top = False; # turn off top and left labeling

As for the axes, each matplotlib object has a 'handle'. Grid, in this case, is the handle of our gridlines. Therefore, we can change its properties by using the `.` syntax!

## 04_04 Two Dimensions - Other Plots

Besides a map in 2D, we could also plot the 2m temperature averaged over longitudes and grouped and averaged by month in a 2D-plot! We will do the commands step by step underneath. This is a good revision of the things we learned in the last chapter.

In [ ]:
t2m_avg = ds.t2m.mean(dim = 'longitude'); #average over longitudes
t2m_grouped = t2m_avg.groupby('time.month').mean(dim = 'time') - 273.15; #groupby, average over month + convert into °C
t2m_grouped.T.plot() # the "T" in here means "transpose" and exchanges the axes that are plotted:
                                            # without the T, latitude would be on the x-axis, month on the y-axis

This type of plot is called **Hovmöller Diagram** and is used very often in climatology. We see the distribution of the 2m temperature over the year at different latitudes. 

## 04_05 Control on 2D Plots

### 04_05_01 Discrete Levels

Smooth color tables like the above are nice and appealing, but the human eye is not trained to see such small differences in colors. For example. it would be quite difficult to tell which temperature does the coast of Peru have (above 280K? or below?). Sometimes, discrete levels are the way to go: 

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2tavg.plot(ax=ax, transform=ccrs.PlateCarree(), levels = [240, 260, 280, 285, 290, 295, 300]) 
ax.coastlines(); ax.gridlines();

You can either specify the keyword argument **levels** with a list of values that determine the different levels (as above) or give a single value. This value determines how many levels you want to create (e.g. levels = 12). Furthermore, we can set the minimum/maximum value that we want to represent in our colorbar by specifying the `vmin` / `vmax` keyword argument in our plot function.
Let's try it:

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2tavg.plot(ax=ax, transform=ccrs.PlateCarree(), levels = 7, vmin = 250, vmax = 310) 
ax.coastlines(); ax.gridlines();

Outliers in the data often have an extreme effect on the output of the plot: E.g. we have a outlier in 2m temperature data of 500K. This means, that our automatically generated colorbar will go up until 500 K! This washes out the real observed pattern and makes it hard to distinguish between values that are reasonable. An easy way to visualize the data without the outliers is to pass the parameter `robust=True`. This will use the 2nd and 98th percentiles of the data to compute the color limits.

### 04_05_02 Colormaps

Python automatically choses the colorbar for our plots, that the program thinks would fit best. So if we now change our 2m temperature from Kelvin to °C, python will chose another colorbar:

In [ ]:
t2ctavg = t2tavg - 273.15

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2ctavg.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines();

We notice, that python decided for a two sided color scheme! This seams reasonable, since we now have values bigger and smaller than zero. Still, the plot is not quite perfect yet. We also notice, that with this color scheme the data range is mostly dictated by very cold temperatures in Antarctica. Let's change this first, by introducing levels and min/max values for the colorbar.

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2ctavg.plot(ax=ax, transform=ccrs.PlateCarree(), levels = 7, vmin = -20, vmax = 40) 
ax.coastlines(); ax.gridlines();

Well, now we have a better color distribution, but python jumped back into the default colormap! That's not what we actually wanted. So it is best, if we specify what our colormap should look like. We can do this, by passing the `cmap = 'your_colormap'` to the plot method. 

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2ctavg.plot(ax=ax, transform=ccrs.PlateCarree(), levels = 7, vmin = -30, vmax = 30, cmap = 'inferno') 
ax.coastlines(); ax.gridlines();

There are a lot of different colormaps available. Look [here](https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html), to get an overview and chose the most appropriate one for your plots!

As a general overwiev:

* **precipitation data**: `blues`, `YlGnBu` work quite good

* **terrain heights**: `terain`

* **temperature**: `inferno`, `hot_r`, `coolwarm` 

* **wind data**: e.g. `cool` 

This is just a suggestion, play around to find the colormap that fits your needs best.

Furthermore, we can control the settings of the **colorbar** that is displayed, by passing  `cbar_kwargs={}` arguments to the plot method. E.g. change the labeling of the cbar: `cbar_kwargs={'label':'my_label'}`.

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2ctavg.plot(ax=ax, transform=ccrs.PlateCarree(), levels = 7, vmin = -30, vmax = 30, cmap = 'inferno', 
             cbar_kwargs={'label': '°C'}) 
ax.coastlines(); ax.gridlines();

If we create a irregularly spaced level list, we can adjust the colorbar so that the the spacing will be proportional! Therefore, we need to specify the `cbar_kwargs={'spacing':'proportional'}`.

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2ctavg.plot(ax=ax, transform=ccrs.PlateCarree(), levels = [-30,-20, 0, 20, 30], cmap = 'inferno', 
             cbar_kwargs={'label': '°C', 'spacing':'proportional'}) 
ax.coastlines(); ax.gridlines();

Another useful thing is, that we can also determine, how many ticks are displayed in the colorbar. By default, python sets the ticks according to your `levels`. 

Anyway, we can also tick every second value of our `levels`:

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2ctavg.plot(ax=ax, transform=ccrs.PlateCarree(), levels = 7, vmin = -30, vmax = 30, cmap = 'inferno', 
             cbar_kwargs={'label': '°C', 'ticks': [-20,0,20]}) 
ax.coastlines(); ax.gridlines();

As for 1D plots, you can set a title for the plot, or give a different figure size:

In [ ]:
fig = plt.figure(figsize = [10, 3])
ax = plt.axes(projection=ccrs.Robinson())
t2ctavg.plot(ax=ax, transform=ccrs.PlateCarree(), levels = 7, vmin = -30, vmax = 30, cmap = 'inferno', 
             cbar_kwargs={'label': '°C'})  
ax.coastlines(); ax.gridlines();
ax.set_title('Averaged 2 meter temperature');


For more information about plotting parameters, a look at xarray's [documentation](http://xarray.pydata.org/en/stable/generated/xarray.plot.pcolormesh.html) might be helpful.


### 04_05_03 Plot types

### Imshow

There are several different possibilities how to visualize a plot 2D plot. We recall the Hovmöller Plot from above:

In [ ]:
t2m_avg = ds.t2m.mean(dim = 'longitude'); #average over longitudes
t2m_grouped = t2m_avg.groupby('time.month').mean(dim = 'time') - 273.15; #groupby, average over month + convert into °C
t2m_grouped.T.plot(); # the "T" in here means "transpose" and exchanges the axes that are plotted:

And notice the following: This plot is quite  "pixellized"! Xarray's method of choice to display 2d data is to represent it as if it was an "image". This type of plot it called `imshow` in python and is the default plot type for a 2D plot. With `imshow`, we can also show images of pictures:


In [ ]:
image = plt.imread('xarraylogo.png') # read the image

plt.imshow(image) #create an axes
plt.axis('off')  # get rid of x-axis and y-axis

### Pcolormesh

This creates a pseudocolor plot with a non-regular rectangular grid. Pcolormesh does not
interpolate, but shows the value in each quadrilateral as a block of
solid color, so it is more like an image compared to contourf (see underneath).



In [ ]:
t2m_avg = ds.t2m.mean(dim = 'longitude');
t2m_grouped = t2m_avg.groupby('time.month').mean(dim = 'time') - 273.15; 
t2m_grouped.T.plot.pcolormesh(levels = 20); 

There are data sets and situations for which pcolormesh as well as contourf can be appropriate, and other cases where one is clearly better than the other.  For a somewhat noisy field, pcolormesh is usually better; it allows one to see the signal and the noise, and let one's eye pick out the former.  Isolated extreme values are represented better with
pcolormesh.  For smoother fields and for seeing large-scale structure, contourf may be more appropriate. 

### Contourf

Contourf creates a filled contour plot containing the isolines of the temperature data. Python automatically selects the contour lines to display. 

In [ ]:
t2m_avg = ds.t2m.mean(dim = 'longitude');
t2m_grouped = t2m_avg.groupby('time.month').mean(dim = 'time') - 273.15; 
t2m_grouped.T.plot.contourf(); 

This does not look very useful. As for the colormaps, you can change the number of isolines displayed by specifying the keyword argument `levels` and change the colormap with `cmap`, give minima and maxima values,...:


In [ ]:
t2m_grouped.T.plot.contourf(levels = np.arange(-50, 21, 10), cmap = 'inferno', vmin = -50, vmax = 25); 


**Notice**: This time we created the levels by creating a numpy array! The array goes from -60 to 40 (last value not inclueded) in steps of 10. 

We can also set the `extend` of the plot. This argument determines, whether the values that lie outside the range defined in `lebels` are to be colored or not. You can set `'extend'` to `'neither'` (colors none of the values outside the levels range), `'both'` (colors both values: above and below the levels range), `'max'` (colors only the values above) or `'min'` (colors only the values below. By default, the extend argument is adapts to your levels argument. If there are any values that exceed the levels range, they will be colored. If there are no, the colorbar will only display the range of the levels argument.

Let's try the different settings:

In [ ]:
fig = plt.figure(figsize=(18, 10))
ax = plt.subplot(2,2,1)
t2m_grouped.T.plot.contourf(levels = np.arange(-50, 21, 10), cmap = 'inferno', extend = 'neither'); 
ax.set_title('extend = neither');
ax = plt.subplot(2,2,2)
t2m_grouped.T.plot.contourf(levels = np.arange(-50, 21, 10), cmap = 'inferno', extend = 'both'); 
ax.set_title('extend = both');
ax = plt.subplot(2,2,3)
t2m_grouped.T.plot.contourf(levels = np.arange(-50, 21, 10), cmap = 'inferno', extend = 'min'); 
ax.set_title('extend = min');
ax = plt.subplot(2,2,4)
t2m_grouped.T.plot.contourf(levels = np.arange(-50, 21, 10), cmap = 'inferno', extend = 'max'); 
ax.set_title('extend = max');


### Contour

This will create a plot containing the contour lines of the given data, in our case 2m temperature. 

In [ ]:
t2m_avg = ds.t2m.mean(dim = 'longitude');
t2m_grouped = t2m_avg.groupby('time.month').mean(dim = 'time') - 273.15; 
t2m_grouped.T.plot.contour(levels = np.arange(-48,25,8)); 

Here, it does not seem to make a lot of sense. Anyway, e.g. for a pressure distribution in x-y plane it can become a useful feature. The parameter setting works as for the above plot types. Still, we can lern what features this plot has and how we can modify them. The bad thing right now is, that our contourlines do not have any labels, so we do actually not have any clue about the values of the displayed data. 

We can change this by creating some labeling! This works with the `plt.clabel(CS, levels, fontsize)`. 

Let's try it:

In [ ]:
CS = t2m_grouped.T.plot.contour(levels = np.arange(-48,25,8)); #CS is the object that is returned by the contour method!
plt.clabel(CS, np.arange(-48,25,16), fontsize = 'large'); #label every second contourline!

CS is a **positional argument** (MUST be given in order to make the method work. The levels array and fontsize are optional. CS is the object that is returned by contour(). So similar to the handle of an axis, CS is the handle of the created contour-plot! That is a general structure of matplotlib objects! The second argument is a list/array of level values, that should be labeled. The list must be a subset of the given contour levels. If not given, all levels are labeled. And finally fontsize is the size of the labels in points or relative size e.g., 'smaller', 'x-large'. There are more properties that you can modify if you want. [Here's](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.clabel.html) the documentation of `clabel`!

Now that we have labeled our contourlines, we can also modify the lines itself by giving more arguments to the contour method itself!

We can e.g. change the linewidth or the linestyle or force all the contours to be the same color:

In [ ]:
CS = t2m_grouped.T.plot.contour(levels = np.arange(-48,25,8), linewidths = 3, linestyles = 'dashed', colors = 'k'); #CS is the object that is returned by the contour method!
plt.clabel(CS, np.arange(-48,25,16), fontsize = 'large'); #label every second contourline!

`linestyles` can be 'solid', 'dashed', 'dashdot' or 'dotted'. If we do not define any `linestyles`, but force all lines to be the same color, lines with negative values will be dashed by default!

### Contours over a pcolormesh

It is also possible to put e.g. a pcolormesh plot in the same axes (plot) as a contour plot! This simply means, you that python will plot the contourlines over the pcolormesh.

Here is an example:

In [ ]:
ax = plt.axes()
mesh = t2m_grouped.T.plot.pcolormesh(levels = np.arange(-50, 36, 5)); # plot a pcolormesh first
contours = t2m_grouped.T.plot.contour(levels = np.arange(-50, 36, 5), colors = 'k'); # plot contourlines over it
plt.clabel(contours, np.arange(-50,36,10)) # label every second contour line
ax.set_xlim(1,12); # set xlimits of the plot (of the axes ax)

## 04_06 Summary

To summarize what we learned in this and the last chapter, we will now produce a plot with four subplots for the seasonal temperature averages over the african continent in a PlateCarree Projection in the form of a contourf plot. 

We start with selecting africa, grouping by month and averaging over the seasons:

In [ ]:
africa = ds.t2m.sel(latitude = slice(45, -40), longitude = slice(-20,60))
temp_month=africa.groupby('time.month').mean(dim='time')-273.15
temp_djf=temp_month.sel(month=[1, 2, 12]).mean(dim='month')
temp_mam=temp_month.sel(month=[3, 4, 5]).mean(dim='month')
temp_jja=temp_month.sel(month=[6, 7, 8]).mean(dim='month')
temp_son=temp_month.sel(month=[9, 10, 11]).mean(dim='month')


Now produce the plot:

In [ ]:
figur = plt.figure(figsize=(16, 16)) #define a figure of a certain size
ax = plt.subplot(2, 2, 1, projection=ccrs.PlateCarree()) #first subplot
temp_djf.plot.contourf(ax=ax, transform=ccrs.PlateCarree(),levels=np.arange(5, 45, 3), cmap='hot_r', cbar_kwargs={'label':'[°C]'})
# the actual plot with levels, a colormap and a labeled colorbar
ax.coastlines(); #add coastlines
xl = ax.gridlines(draw_labels=True); #add gridlines with labels
xl.xlabels_top = False # no labels at the top
xl.ylabels_right = False # no labels on the right side
ax.add_feature(cartopy.feature.BORDERS, linestyle='--'); # add dashed country borders
ax.set_title('Average T        DJF'); # add a title

ax = plt.subplot(2, 2, 2, projection=ccrs.PlateCarree())
temp_mam.plot.contourf(ax=ax, transform=ccrs.PlateCarree(),levels=np.arange(5, 43, 3), cmap='hot_r', cbar_kwargs={'label':'[°C]'})
ax.coastlines();
xl = ax.gridlines(draw_labels=True);
xl.xlabels_top = False
xl.ylabels_right = False
ax.add_feature(cartopy.feature.BORDERS, linestyle='--');
ax.set_title('Average T        MAM');

ax = plt.subplot(2, 2, 3, projection=ccrs.PlateCarree())
temp_jja.plot.contourf(ax=ax, transform=ccrs.PlateCarree(),levels=np.arange(5, 45, 3), cmap='hot_r', cbar_kwargs={'label':'[°C]'})
ax.coastlines();
xl = ax.gridlines(draw_labels=True);
xl.xlabels_top = False
xl.ylabels_right = False
ax.add_feature(cartopy.feature.BORDERS, linestyle='--');
ax.set_title('Average T       JJA');
                   
ax = plt.subplot(2, 2, 4, projection=ccrs.PlateCarree())
temp_son.plot.contourf(ax=ax, transform=ccrs.PlateCarree(),levels=np.arange(5, 45, 3), cmap='hot_r', cbar_kwargs={'label':'[°C]'})
ax.coastlines();
xl = ax.gridlines(draw_labels=True);
xl.xlabels_top = False
xl.ylabels_right = False
ax.add_feature(cartopy.feature.BORDERS, linestyle='--');
ax.set_title('Average T       SON');